In [29]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression,Ridge,Lasso
from sklearn.metrics import mean_squared_error, r2_score,mean_absolute_error
from sklearn.model_selection import RandomizedSearchCV
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor,GradientBoostingRegressor,AdaBoostRegressor
from sklearn.neighbors import KNeighborsRegressor
from xgboost import XGBRegressor
import warnings
warnings.filterwarnings('ignore')

In [18]:
df = pd.read_csv('data/StudentsPerformance.csv')
df.head()

,gender,race/ethnicity,parental level of education,lunch,test preparation course,math score,reading score,writing score
0,female,group B,bachelor's degree,standard,none,72,72,74
1,female,group C,some college,standard,completed,69,90,88
2,female,group B,master's degree,standard,none,90,95,93
3,male,group A,associate's degree,free/reduced,none,47,57,44
4,male,group C,some college,standard,none,76,78,75


In [19]:
X = df.drop('math score',axis=1)
y = df['math score']

In [20]:
num_features = X.select_dtypes(exclude='object').columns
cat_features = X.select_dtypes(include='object').columns

from sklearn.preprocessing import OneHotEncoder,StandardScaler
from sklearn.compose import ColumnTransformer

num_transformer = StandardScaler()
cat_transformer = OneHotEncoder()

preprocessor = ColumnTransformer(
    [
    ("OneHotEncoder", cat_transformer, cat_features),
    ("StanderdScaler", num_transformer, num_features),
    ]
)

In [22]:
X=preprocessor.fit_transform(X)

In [23]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [30]:
##Create a Function to Evaluate Model
import numpy as np
def evaluate_model(true, predicted):
    mae = mean_absolute_error(true, predicted)
    mse = mean_squared_error(true, predicted)
    rmse = np.sqrt(mean_squared_error(true, predicted))
    r2_square = r2_score(true, predicted)
    return mae, rmse, r2_square

In [41]:
## Beginning Model Training
models = {
    "Linear Regression": LinearRegression(fit_intercept=False),
    # "Lasso": Lasso(),
    "Ridge": Ridge(solver='sparse_cg',max_iter=200,alpha=1.0)
    # "K-Neighbors Regressor": KNeighborsRegressor(),
    # "Decision Tree": DecisionTreeRegressor(),
    # "Random Forest Regressor": RandomForestRegressor(),
    # "Adaboost Regressor":AdaBoostRegressor(),
    # "Graident Boost": GradientBoostingRegressor(),
    # "Xg boost":XGBRegressor(),
}

for i in range(len(list(models))):
    model = list(models.values())[i]
    model.fit(X_train, y_train) # Train model

    # Make predictions
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)
    
    # Evaluate Train and Test dataset
    model_train_mae , model_train_rmse, model_train_r2 = evaluate_model(y_train, y_train_pred)

    model_test_mae , model_test_rmse, model_test_r2 = evaluate_model(y_test, y_test_pred)

    
    print(list(models.keys())[i])
    
    print('Model performance for Training set')
    print("- Root Mean Squared Error: {:.4f}".format(model_train_rmse))
    print("- Mean Absolute Error: {:.4f}".format(model_train_mae))
    print("- R2 Score: {:.4f}".format(model_train_r2))

    print('----------------------------------')
    
    print('Model performance for Test set')
    print("- Root Mean Squared Error: {:.4f}".format(model_test_rmse))
    print("- Mean Absolute Error: {:.4f}".format(model_test_mae))
    print("- R2 Score: {:.4f}".format(model_test_r2))
    
    print('='*35)
    print('\n')

Linear Regression
Model performance for Training set
- Root Mean Squared Error: 5.3310
- Mean Absolute Error: 4.2776
- R2 Score: 0.8739
----------------------------------
Model performance for Test set
- Root Mean Squared Error: 5.3951
- Mean Absolute Error: 4.2092
- R2 Score: 0.8804


Ridge
Model performance for Training set
- Root Mean Squared Error: 5.3233
- Mean Absolute Error: 4.2650
- R2 Score: 0.8743
----------------------------------
Model performance for Test set
- Root Mean Squared Error: 5.3904
- Mean Absolute Error: 4.2111
- R2 Score: 0.8806




In [38]:
## Hyperparameter Training
linear_params = {
    'fit_intercept':[True,False],
}
ridge_params = {
    'alpha':[0.1,1.0,10,100,200,500,1000],
    'solver':['auto','svd','cholesky','lsqr','sparse_cg','sag','saga','lbfgs'],
    'max_iter':[100,200,500,1000,2000],
}
# adaboostParams = {
#     'n_estimators':[50,60,70,80,90],
#     'algorithm':['SAMME','SAMME.R']
# }
# gradient_params = {
#     'loss':['log_loss','deviance','exponential'],
#     'criterion':['friedman_mse','squared_error','mse'],
#     'min_samples_split':[2,8,15,20],
#     'n_estimators':[100,200,500,1000],
#     'max_depth':[5,8,15,None,10]
# }
# xg_params = {
#      "learning_rate":[0.1,0.01],
#      "max_depth":[5,8,12,20,30],
#      "n_estimators":[100,200,300],
#      "colsample_bytree":[0.5,0.8,1,0.3,0.4]


In [39]:
# Models list for Hyperparameter tuning
randomcv_models = [
                   ("LinearRegression", LinearRegression(), linear_params),
                   #("ADA",AdaBoostClassifier(),adaboostParams),
                    # ("Gradient Boost",GradientBoostingClassifier(),gradient_params),
                    ("Ridge",Ridge(),ridge_params),
                   ]

In [40]:

model_param = {}
for name, model, params in randomcv_models:
    random = RandomizedSearchCV(estimator=model,
                                   param_distributions=params,
                                   n_iter=100,
                                   cv=3,
                                   verbose=2,
                                   n_jobs=-1)
    random.fit(X_train, y_train)
    model_param[name] = random.best_params_

for model_name in model_param:
    print(f"---------------- Best Params for {model_name} -------------------")
    print(model_param[model_name])

Fitting 3 folds for each of 2 candidates, totalling 6 fits
Fitting 3 folds for each of 100 candidates, totalling 300 fits
---------------- Best Params for LinearRegression -------------------
{'fit_intercept': False}
---------------- Best Params for Ridge -------------------
{'solver': 'sparse_cg', 'max_iter': 200, 'alpha': 1.0}
